In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
def rho_red_light(x, rho_max):
    rho = rho_max * numpy.ones_like(x)
    mask = numpy.where(x < 3.0)
    rho[mask] = 0.3 * rho_max
    return rho

In [3]:
nx = 81  
L = 4.0  
dx = L / (nx - 1) 
nt = 40  
rho_max = 10.0  
u_max = 1.0  
x = numpy.linspace(0.0, L, num=nx)
rho0 = rho_red_light(x, rho_max)

In [4]:
def maccormack(rho0, nt, dt, dx, bc_values, *args):
    rho_hist = [rho0.copy()]
    rho = rho0.copy()
    rho_star = rho.copy()
    for n in range(nt):
        F = flux(rho, *args)
        rho_star[1:-1] = (rho[1:-1] -
                          dt / dx * (F[2:] - F[1:-1]))
        F = flux(rho_star, *args)
        rho[1:-1] = 0.5 * (rho[1:-1] + rho_star[1:-1] -
                           dt / dx * (F[1:-1] - F[:-2]))
        rho[0] = bc_values[0]
        rho[-1] = bc_values[1]
        rho_hist.append(rho.copy())
    return rho_hist

In [5]:
sigma = 0.6
dt = sigma * dx / u_max
def flux(rho, u_max, rho_max):
    F = rho * u_max * (1.0 - rho / rho_max)
    return F
rho_hist = maccormack(rho0, nt, dt, dx, (rho0[0], rho0[-1]),
                      u_max, rho_max)

In [6]:
import ipywidgets

In [7]:
def interactive_plot(x,rho_hist):
    nt = len(rho_hist)-1
    nt_slider = ipywidgets.IntSlider(value=0,min=0,max=nt,step=1,description='time step')
    w = ipywidgets.interactive(plot,n=nt_slider,x=ipywidgets.fixed(x),rho_hist=ipywidgets.fixed(rho_hist))
    return w

In [8]:
def plot(n,x,rho_hist):
    pyplot.figure()
    pyplot.grid()
    pyplot.title('time step'.format(n))
    pyplot.xlabel('road')
    pyplot.ylabel('rho')
    pyplot.plot(x,rho_hist[n])
    pyplot.xlim(x[0],x[-1])
    pyplot.ylim(1.0,12.0)
    pyplot.show
interactive_plot(x,rho_hist)

interactive(children=(IntSlider(value=0, description='time step', max=40), Output()), _dom_classes=('widget-in…

In [9]:
def jacobian(rho, u_max, rho_max):
    J = u_max * (1.0 - 2.0 * rho / rho_max)
    return J

In [10]:
def lax_wendroff(rho0, nt, dt, dx, bc_values, *args):
    rho_hist = [rho0.copy()]
    rho = rho0.copy()
    for n in range(nt):
        # Compute the flux.
        F = flux(rho, *args)
        # Compute the Jacobian.
        J = jacobian(rho, *args)
        # Advance in time using Lax-Wendroff scheme.
        rho[1:-1] = (rho[1:-1] -
                     dt / (2.0 * dx) * (F[2:] - F[:-2]) +
                     dt**2 / (4.0 * dx**2) *
                     ((J[1:-1] + J[2:]) * (F[2:] - F[1:-1]) -
                      (J[:-2] + J[1:-1]) * (F[1:-1] - F[:-2])))
        # Set the value at the first location.
        rho[0] = bc_values[0]
        # Set the value at the last location.
        rho[-1] = bc_values[1]
        # Record the time-step solution.
        rho_hist.append(rho.copy())
    return rho_hist

In [11]:
def interactive_plot(x,rho_hist):
    nt = len(rho_hist)-1
    nt_slider = ipywidgets.IntSlider(value=0,min=0,max=nt,step=1,description='time step')
    w = ipywidgets.interactive(plot,n=nt_slider,x=ipywidgets.fixed(x),rho_hist=ipywidgets.fixed(rho_hist))
    return w
interactive_plot(x,rho_hist)

interactive(children=(IntSlider(value=0, description='time step', max=40), Output()), _dom_classes=('widget-in…

In [12]:
def lax_friedrichs(rho0, nt, dt, dx, bc_values, *args):
    rho_hist = [rho0.copy()]
    rho = rho0.copy()
    for n in range(nt):
        # Compute the flux.
        F = flux(rho, *args)
        # Advance in time using Lax-Friedrichs scheme.
        rho[1:-1] = (0.5 * (rho[:-2] + rho[2:]) -
                     dt / (2.0 * dx) * (F[2:] - F[:-2]))
        # Set the value at the first location.
        rho[0] = bc_values[0]
        # Set the value at the last location.
        rho[-1] = bc_values[1]
        # Record the time-step solution.
        rho_hist.append(rho.copy())
    return rho_hist

In [13]:
def interactive_plot(x,rho_hist):
    nt = len(rho_hist)-1
    nt_slider = ipywidgets.IntSlider(value=0,min=0,max=nt,step=1,description='time step')
    w = ipywidgets.interactive(plot,n=nt_slider,x=ipywidgets.fixed(x),rho_hist=ipywidgets.fixed(rho_hist))
    return w
interactive_plot(x,rho_hist)

interactive(children=(IntSlider(value=0, description='time step', max=40), Output()), _dom_classes=('widget-in…